In [1]:
from flask import Flask, Response
from flask_cors import CORS
import cv2
import mediapipe as mp
import numpy as np
import pickle
import threading


# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Load trained model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5)

# Labels dictionary for predictions
labels_dict = {i: chr(65 + i) for i in range(26)}

# List to store detected letters and form the sentence
sentence = []
current_letter = "?"  # Track the current detected letter
final_sentence = ""  # Store the printed sentence


def generate_frames():
    global current_letter, sentence, final_sentence  # Access global variables
    cap = cv2.VideoCapture(0)  # Open camera

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB for MediaPipe processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        # Default to "?" if no hand is detected
        current_letter = "?"

        # If hand landmarks are detected, predict the character
        if results.multi_hand_landmarks:
            landmarks = results.multi_hand_landmarks[0]
            data_aux = []

            x_, y_ = [lm.x for lm in landmarks.landmark], [lm.y for lm in landmarks.landmark]
            for lm in landmarks.landmark:
                data_aux.append(lm.x - min(x_))
                data_aux.append(lm.y - min(y_))

            # Ensure proper data length for model input
            data_aux = data_aux[:42] + [0] * max(0, 42 - len(data_aux))
            prediction = model.predict([np.asarray(data_aux)])
            current_letter = labels_dict.get(int(prediction[0]), "?")

        # Add the current forming sentence, detected letter, and final sentence to the video frame
        cv2.putText(frame, f"Current Letter: {current_letter}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3)
        cv2.putText(frame, "Forming Sentence: " + "".join(sentence), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        cv2.putText(frame, "Final Sentence: " + final_sentence, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Sign Language Detection", frame)

        # Listen for key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('k'):  # Save the detected letter
            if current_letter != "?":
                sentence.append(current_letter)
                print(f"Saved Letter: {current_letter}")
        elif key == ord('s'):  # Add a space
            sentence.append(" ")
            print("Added Space.")
        elif key == ord('d'):  # Delete the last letter (Backspace functionality)
            if sentence:
                deleted_letter = sentence.pop()
                print(f"Deleted Letter: {deleted_letter}")
            else:
                print("No letter to delete.")
        elif key == ord('p'):  # Print the final sentence on screen and reset `sentence`
            final_sentence = "".join(sentence)
            print(f"Final Sentence Printed: {final_sentence}")
            sentence = []  # Clear the forming sentence after printing
        elif key == ord('q'):  # Quit the loop and stop the program
            print("Exiting...")
            
            break

    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')

def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Run Flask app in a separate thread
def run_app():
    app.run(debug=True, use_reloader=False)

flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# Run video capture and key detection in the main thread
# generate_frames()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
import numpy as np
import cv2
import os
from werkzeug.utils import secure_filename
import nest_asyncio  # Required for Jupyter notebooks

# Allow Flask to run smoothly inside Jupyter Notebook
nest_asyncio.apply()

# Initialize Flask app and enable CORS
app = Flask(__name__)
CORS(app)

# Load the trained model
model = tf.keras.models.load_model("./sign_language_model.h5")  # Check model path

# Class labels for prediction
class_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9',
                'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
                'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                'U', 'V', 'W', 'X', 'Y', 'Z']

# Function to preprocess images for prediction
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    img = cv2.resize(img, (64, 64))  # Resize as per model input shape
    img = img / 255.0  # Normalize image (pixel values between 0 and 1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# API route to handle image uploads and predictions
@app.route('/predict', methods=['POST'])
def predict_sign():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    if file:
        # Save the uploaded file temporarily
        filename = secure_filename(file.filename)
        file_path = os.path.join("temp", filename)
        os.makedirs("temp", exist_ok=True)  # Ensure the temp folder exists
        file.save(file_path)

        # Preprocess the saved image and make a prediction
        processed_img = preprocess_image(file_path)
        prediction = model.predict(processed_img)
        predicted_class = class_labels[np.argmax(prediction)]

        # Clean up: remove the temporary image file
        os.remove(file_path)

        # Return the predicted class in the response
        return jsonify({"predicted_class": predicted_class})

    return jsonify({"error": "File upload failed"}), 500


# Run Flask app with Jupyter compatibility
app.run(debug=True, use_reloader=False, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
